In [1]:
%matplotlib inline
# %matplotlib is a magic function in IPython. I'll quote the relevant documentation here for you to read for convenience:
# IPython has a set of predefined ‘magic functions’ that you can call with a command line style syntax. There are two kinds of magics, line-oriented and cell-oriented. Line magics are prefixed with the % character and work much like OS command-line calls: they get as an argument the rest of the line, where arguments are passed without parentheses or quotes. Lines magics can return results and can be used in the right hand side of an assignment. Cell magics are prefixed with a double %%, and they are functions that get as an argument not only the rest of the line, but also the lines below it in a separate argument.
# %matplotlib inline sets the backend of matplotlib to the 'inline' backend:
# With this backend, the output of plotting commands is displayed inline within frontends like the Jupyter notebook, directly below the code cell that produced it. The resulting plots will then also be stored in the notebook document.
# When using the 'inline' backend, your matplotlib graphs will be included in your notebook, next to the code. It may be worth also reading How to make IPython notebook matplotlib plot inline for reference on how to use it in your code.
# If you want interactivity as well, you can use the nbagg backend with %matplotlib notebook (in IPython 3.x), as described here.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from pygmid import Lookup as lk
# import Lookup as lk
# Importing modules from parent folder or subdirectory
# import src.pygmid.Lookup as lut
from Lookup import Lookup as lut
import os
from scipy.interpolate import make_interp_spline
from scipy.interpolate import interp1d
from scipy.interpolate import PchipInterpolator

from IPython.core.display import Image
# filepath_Book_65nm_figs = 'C:\\Users\\jruib\\OneDrive\\Documents\\Systematic_Analog_Design\\LUTs_65nm\\plots_LUTs_python\\Book_Chap1\\'
# fig_1_1_65nm = filepath_Book_65nm_figs + 'Book_Fig_1_1.png'
# Image(fig_1_1_65nm, width=800, height=800)

`Introduction`
============================================
It is well known that the square-law MOS model is plagued by several limitations, especially when it comes to short-channel transistors: ​​ Modern MOSFETs are impaired by numerous mobility degradation effects, related to their short channel length, thin gate oxide and their generally more complex structure and doping profiles. In strong inversion, with gate overdrive voltages (VGS – VT) of several hundred millivolts, the error in the transconductance predicted by square-law models with constant parameters is of the order of 20–60%.  
  
In moderate inversion, with gate overdrive voltages below 150 mV, the square-law model breaks down altogether and it may be in error by a factor of two or even more. This deficiency applies to all MOSFETs, regardless of channel length. However, the issue has become more pronounced with short channel devices, since moderate inversion represents a design “sweet spot” for a variety of circuits in these technologies [3]–[5].  
  
In weak inversion (subthreshold operation), the current flows by diffusion (like in a BJT) and the square-law model must be replaced with an exponential I–V relationship.

**`Before proceeding any further let's define the technology used below`**  
*By Default the Book 65nm models are used as to compare with the plots shown in the Book.*


In [ ]:
filepath_dir = os.getcwd()
print(f"filepath_dir = {filepath_dir}")
str_technology = input("Please choose desired technology in nm: 65 (def), 180, 40, 28, 22) ")
str_vt = input('Please insert Vth flavor (svt, lvt, hvt): ')
str_mos_lib = input('Please insert MOS process (tt, ss or ff): ')
    
if str_technology == '180':
    filepath_dir = 'C:\\Users\\jruib\\OneDrive\\Documents\\Systematic_Analog_Design\\LUTs_180nm\\'
    device1 = f'180nch_{str_vt}_mac_{str_mos_lib}_lib.mat'
    device2 = f'180pch_{str_vt}_mac_{str_mos_lib}_lib.mat'
elif str_technology == '40':
    filepath_dir = 'C:\\Users\\jruib\\OneDrive\\Documents\\Systematic_Analog_Design\\LUTs_40nm\\'
    device1 = f'40nch_{str_vt}_mac_{str_mos_lib}_lib.mat'
    device2 = f'40pch_{str_vt}_mac_{str_mos_lib}_lib.mat'
elif str_technology == '28':
    filepath_dir = 'C:\\Users\\jruib\\OneDrive\\Documents\\Systematic_Analog_Design\\LUTs_28nm\\'
    device1 = f'28nch_{str_vt}_mac_{str_mos_lib}_lib.mat'
    device2 = f'28pch_{str_vt}_mac_{str_mos_lib}_lib.mat'
elif str_technology == '22':
    filepath_dir = 'C:\\Users\\jruib\\OneDrive\\Documents\\Systematic_Analog_Design\\LUTs_22nm\\'
    device1 = f'22nch_{str_vt}_mac_{str_mos_lib}_lib.mat'
    device2 = f'22pch_{str_vt}_mac_{str_mos_lib}_lib.mat'
else:
    filepath_dir = 'C:\\Users\\jruib\\OneDrive\\Documents\\Systematic_Analog_Design\\LUTs_65nm\\'
    device1 = f'65nch_{str_vt}_mac_{str_mos_lib}_lib.mat'
    device2 = f'65pch_{str_vt}_mac_{str_mos_lib}_lib.mat'

filepath_dir_fig = filepath_dir + 'plots_LUTs_python\\Book_Chap1'
if not os.path.exists(filepath_dir_fig):
    # if the folder directory is not present then create it.
    print("Creating figures/plots directory.")
    os.makedirs(filepath_dir_fig)


### **`Chosen devices`**  

In [ ]:
print(f"device1 = {device1}")
device1_LUT = filepath_dir+device1
print(f"device1_LUT = {device1_LUT}")

print(f"device2 = {device2}")
device2_LUT = filepath_dir+device2
print(f"device2_LUT = {device2_LUT}")
        
NCH = lut(device1_LUT)  # load MATLAB data into pygmid lookup object
PCH = lut(device2_LUT)  # load MATLAB data into pygmid lookup object       

In [ ]:
str_save_Figs = input('Would you like to save the following Figures: (yes/no)')
if str_save_Figs == 'yes':
    save_Figs = True
    dpi = 600
else:
    save_Figs = False


In [ ]:
VDSs = NCH['VDS']       # lookup object has pseudo-array access to data
W_lut = NCH['W']
L_lut = NCH['L']
L_Min = min(L_lut)
dev_Temp = NCH['TEMP']
VGSs = np.arange(0.4, 0.6, 0.05)
VGS_lut = NCH['VGS']
VBS_lut = NCH['VSB']

print(f"***********************************************")
print(f"VDSs = {VDSs}")
print(f"***********************************************")
print(f"W_LUT = {W_lut}")
print(f"***********************************************")
print(f"L_LUT = {L_lut}")
print(f"***********************************************")
print(f"VGS_LUT = {VGS_lut}")
print(f"***********************************************")
print(f"VBS_LUT = {VBS_lut}")
print(f"***********************************************")
print(f"dev_Temp = {dev_Temp}")
print(f"***********************************************")

In [ ]:
def XTRACT(dev, L=0.1, VDS=0.6, VSB=0, rho= 0.6):
    # % EKV param identification algor =====================================
    # % L, VSB are scalars
    # % VDS scalar or column vector
    # % rho reference        (optional scalar)
    # XTRACT.m
    # % rho = .6; dev = nch; VSB = 0; L = .1; VDS = (.1:.1:.8)';
    # NCH = lut(file_LUT)  # load MATLAB data into pygmid lookup object
    print("")
    print(f"Entered XTRACT function(), with input args:")
    print(f"L = {L}")
    print(f"VDS = {VDS}")
    print(f"VSB = {VSB}")
    print(f"rho = {rho}")

    VDSs = dev['VDS']       # lookup object has pseudo-array access to data
    W_lut = dev['W']
    L_lut = dev['L']
    L_Min = min(L_lut)
    dev_Temp = dev['TEMP']
    VGS_lut = dev['VGS']
    VBS_lut = dev['VSB']

    # print(f"VDSs = {VDSs}")
    print(f"W_LUT = {W_lut}")
    print(f"L_LUT_min = {L_Min}")
    # print(f"L_LUT = {L_lut}")
    # print(f"VGS_LUT = {VGS_lut}")
    # print(f"VBS_LUT = {VBS_lut}")
    print(f"dev_Temp = {dev_Temp}")
    print("")


    # % data ================
    UT  = .0259*dev_Temp/300
    print(f"Thermal Voltage for device, UT = {UT*1000} mV.")
    UDS = np.arange(0.025, 1.2+0.025, 0.025)
    print(f"UDS = {UDS}")
    # print(f"UDS = {UDS.T}")
    # # x = np.array(np.squeeze(np.transpose(x, (1, 0, 2, 3))))
    # UDS_T = np.transpose(UDS)
    # print(f"UDS = {UDS}")

    # %  find n(UDS) ==============
    gm_ID = NCH.look_up('GM_ID',VDS=UDS.T, VSB=VSB, L=L)
    np.ndim(gm_ID)
    np.shape(gm_ID)

    # Changes shape!!
    # gm_ID = gm_ID[~np.isnan(gm_ID)]
    print(gm_ID[:,0]) # All rows and Column 0
    print(gm_ID[0,:]) # All columns and Row 0
    print(np.isnan(gm_ID))
    gm_ID = gm_ID[:,1:] # Remove column 0 from Array
    print(np.isnan(gm_ID))
    print(gm_ID[:,0]) # All rows and Column 0

    # ID = NCH.look_up('ID',VDS=UDS, VSB=0.1,L=L_Min)
    # Number of dimensions of the NumPy array: ndim
    # Shape of the NumPy array: shape
    # Size of the NumPy array: size
    # Size of the first dimension of the NumPy array: len()

    # print(f"*************************************")
    # print(f"Test numpy array manipulation")
    # a = np.array([[200,300,400],[100,500,300]])
    # print(f"array = {a}")
    # # array([[200, 300, 400],
    #     #    [100,  50, 300]])
    # print(f"array Transpose = {a.T}")
    # # array([[200, 100],
    # #        [300,  50],
    # #        [400, 300]])
    # print(f"Array dimensions {np.ndim(a)}")  # --> 2
    # print(f"Array dimensions {np.shape(a)}")  # --> (2,3) or 2 rows and 3 columns
    # maxindex = a.argmax()
    # print(f"Array Maximum value index = {maxindex}")  # --> 2
    # max_row_col = np.unravel_index(maxindex, a.shape)
    # print(f"Array Maximum value row & col = {max_row_col}")  # --> (1, 1)
    # print(f"Array Maximum value = {a[max_row_col]}")  # --> 500
    # print(f"*************************************")

    print(f"*************************************")
    gm_ID_shape = np.shape(gm_ID)
    gm_ID_dim = np.ndim(gm_ID)
    print(f"Gm_ID dimensions {gm_ID_dim}")  # --> 2
    print(f"Gm_ID dimensions {gm_ID_shape}")  # --> (2,3) or 2 rows and 3 columns
    gm_ID_maxindex = gm_ID.argmax()
    print(f"Gm_ID Maximum value index = {gm_ID_maxindex}")  # --> 2
    gm_ID_max_row_col = np.unravel_index(gm_ID_maxindex, gm_ID_shape)
    print(f"Gm_ID Maximum value row & col = {gm_ID_max_row_col}")  # --> (1, 1)
    print(f"Gm_ID Maximum value = {gm_ID[gm_ID_max_row_col]}")  # --> 500
    gm_ID_max_row_vals = gm_ID[gm_ID_max_row_col[0],:] # Copy entire row where max value is located
    M = gm_ID_max_row_vals.T
    nn = 1/(M*UT)

    # % find VT(UDS) ============
    q  = 1/rho - 1
    i  = q**2 + q
    VP = UT*(2*(q-1) + np.log(q))

    gmIDref = rho*M
    VGS_interp = []
    # for k in range(0,len(UDS)+1):
    for k in range(0,len(UDS)-1):
        # VGS(k,1) = interp1(gm_ID(:,k),dev.VGS,gmIDref(k),'pchip'); # MATLAB
        # VGS[k,1] = interp1d(gm_ID[:,k],VGS_lut)(gmIDref[k])
        set_interp = interp1d(gm_ID[:,k],VGS_lut, kind='cubic')
        VGS_interp.append(set_interp(gmIDref[k]))
    print(f"*************************************")
    VGS_interp = np.array(VGS_interp)

    Vth = VGS_interp - nn*VP

    # %find JS(UDS) ===============
    # Js = np.diag(lookup(dev,'ID_W','GM_ID',gmIDref,'VDS',UDS,'VSB',VSB,'L',L))/i;
    # Js = np.diag(NCH.look_up('ID_W', GM_ID=gmIDref, VDS=UDS, VSB=0, L=L_Min))/i
    Js_interp = np.diag(NCH.look_up('ID_W', GM_ID=gmIDref, VDS=UDS[1:], VSB=VSB, L=L))/i

    # % DERIVATIVES ===============
    # UDS1 = .5*(UDS(1:end-1) + UDS(2:end));
    # UDS2 = .5*(UDS1(1:end-1) + UDS1(2:end));
    # UDS1 = .5*(UDS[0:-1] + UDS[1:])
    # UDS2 = .5*(UDS1[0:-1] + UDS1[1:])
    # diffUDS = np.diff(UDS)
    # diffUDS1 = np.diff(UDS1)

    UDS1 = .5*(UDS[1:-1] + UDS[2:])
    UDS2 = .5*(UDS1[0:-1] + UDS1[1:])
    diffUDS = np.diff(UDS[1:])
    diffUDS1 = np.diff(UDS1)

    # % subthreshold slope ============
    diff1n = np.diff(nn)/diffUDS
    diff2n = np.diff(diff1n)/diffUDS1

    # % threshold voltage =============
    diff1Vth = np.diff(Vth)/diffUDS
    diff2Vth = np.diff(diff1Vth)/diffUDS1

    # % log specific current ============
    diff1logJs = np.diff(np.log(Js_interp))/diffUDS
    diff2logJs = np.diff(diff1logJs)/diffUDS1

    # % n(VDS), VT(VDS) , JS(VDS) ===========
    # n  = interp1(UDS,nn,VDS,'pchip');
    n = interp1d(UDS[1:],nn, kind='cubic')(VDS)
    # VT = interp1(UDS,Vth,VDS,'pchip');
    VT = interp1d(UDS[1:],Vth, kind='cubic')(VDS)
    # JS = interp1(UDS,Js,VDS,'pchip');
    Js = interp1d(UDS[1:],Js_interp, kind='cubic')(VDS)

    # d1n = interp1(UDS1,diff1n,VDS,'pchip');
    # d2n = interp1(UDS2,diff2n,VDS,'pchip');
    d1n = interp1d(UDS1,diff1n, kind='cubic')(VDS)
    d2n = interp1d(UDS2,diff2n, kind='cubic')(VDS)

    # d1VT = interp1(UDS1,diff1Vth,VDS,'pchip');
    # d2VT = interp1(UDS2,diff2Vth,VDS,'pchip');
    d1VT = interp1d(UDS1,diff1Vth, kind='cubic')(VDS)
    d2VT = interp1d(UDS2,diff2Vth, kind='cubic')(VDS)

    # d1logJS = interp1(UDS1,diff1logJs,VDS,'pchip');
    # d2logJS = interp1(UDS2,diff2logJs,VDS,'pchip');
    d1logJS = interp1d(UDS1,diff1logJs, kind='cubic')(VDS)
    d2logJS = interp1d(UDS2,diff2logJs, kind='cubic')(VDS)

    # % OUTPUT ========
    y = [VDS, n, VT, Js, d1n, d1VT, d1logJS, d2n, d2VT, d2logJS]

    return y
    # *********** END OF XTRACT.m **************

**`Greek Symbols`**  
$\alpha$, $\beta$, $\gamma$, $\delta$, $\epsilon$, $\zeta$, $\eta$, $\theta$, $\iota$
$\kappa$, $\lambda$, $\mu$, $\nu$, $\xi$, $\omicron$, $\pi$, $\rho$, $\sigma$, $\tau$  
$\upsilon$, $\phi$, $\chi$, $\psi$, $\omega$,  $\Omega$, $\Delta$, $\Sigma$

In [ ]:
# skip this, this is just to display nice tables.
from itertools import zip_longest
class Table(list):
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            for col in row:
                try:
                    float(col)
                    html.append("<td>%.3f</td>" % col)
                except(ValueError):
                    html.append("<td><b>%s</b></td>" % col)
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)


## **`Fig 1.1 - Current density of a minimum L NCH device Vs Vgs`**  
*Dotted vertical line corresponds to device threshold voltage, $V_{th}$.*



In [ ]:
if save_Figs:
    dpi = 600
    fig_JD_Vgs_1_1 = filepath_dir_fig + f'\\Fig_1_1_JD_Vs_Vgs_max_JD_{np.max(JD)*1e6:.1f}_uA_um_jba'
    plt.savefig(fig_JD_Vgs_1_1 + '.png', dpi = dpi, bbox_inches='tight')

plt.show()
# **************************************************************
# **************************************************************

In [ ]:
print(f"VGS = {VGS} \n")
print(f"JD = {JD} \n")
print(f"VGS ndim = {np.ndim(VGS)} \n")
print(f"VGS shape = {np.shape(VGS)} \n")
print(f"JD ndim = {np.ndim(JD)} \n")
print(f"JD shape = {np.shape(JD)} \n")
JD.max()